### Simple example for performing symbolic regression for a set of points

In [ ]:
from nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
from pathlib import Path
from functools import partial
import torch
from sympy import lambdify
import json

In [ ]:
## Load equation configuration and architecture configuration
import omegaconf
with open('100M/eq_setting.json', 'r') as json_file:
  eq_setting = json.load(json_file)

cfg = omegaconf.OmegaConf.load("100M/config.yaml")

In [ ]:
## Set up BFGS load rom the hydra config yaml
bfgs = BFGSParams(
        activated= cfg.inference.bfgs.activated,
        n_restarts=cfg.inference.bfgs.n_restarts,
        add_coefficients_if_not_existing=cfg.inference.bfgs.add_coefficients_if_not_existing,
        normalization_o=cfg.inference.bfgs.normalization_o,
        idx_remove=cfg.inference.bfgs.idx_remove,
        normalization_type=cfg.inference.bfgs.normalization_type,
        stop_time=cfg.inference.bfgs.stop_time,
    )


In [ ]:
print(cfg.inference.beam_size)

In [ ]:
params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            # beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            beam_size=2
                            )

In [ ]:
weights_path = "100M/100M.ckpt"

In [ ]:
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()

In [ ]:
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [ ]:
# Create points from an equation
number_of_points = 500
n_variables = 1

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(eq_setting["total_variables"])))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "cos(sin(cos(x_1**2)*sin(x_1)))" #Use x_1,x_2 and x_3 as independent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

In [ ]:
print("X shape: ", X.shape)
print("y shape: ", y.shape)

In [ ]:
output = fitfunc(X,y) 

In [ ]:
output

In [ ]:
# target_eq = "cos(x_1**6)*sin(x_1)" #Use x_1,x_2 and x_3 as independent variables
target_eq = "x_1**4"
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)
output = fitfunc(X,y) 
output

# Extension

## Data generation

In [ ]:
# Create points from an equation
number_of_points = 500
n_variables = 1

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(eq_setting["total_variables"])))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "cos(sin(cos(x_1**2)*sin(x_1)))" #Use x_1,x_2 and x_3 as independent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

output = fitfunc(X,y) 
output_all = output["all_bfgs_preds"]
loss_all = output["all_bfgs_loss"]
output_best = output["best_bfgs_preds"]
loss_best = output["best_bfgs_loss"]
print(f"Best prediction: {output_best}")

## GP after BFGS

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms, gp
import sympy as sp
from sympy.utilities.lambdify import lambdify

# Ensure this block runs after obtaining `output_all` from the model
# Create points from an equation
number_of_points = 50
n_variables = 1

# To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"]
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points, len(list(eq_setting["total_variables"]))) * (max_supp - min_supp) + min_supp
X[:, n_variables:] = 0
target_eq = "cos(sin(cos(x_1**2) * sin(x_1)))"  # Use x_1, x_2 and x_3 as independent variables
X_dict = {x: X[:, idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])}
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

output = fitfunc(X, y)
output_all = output["all_bfgs_preds"]
loss_all = output["all_bfgs_loss"]
output_best = output["best_bfgs_preds"]
loss_best = output["best_bfgs_loss"]
print(f"Best prediction: {output_best}")

print(output_all)
equations_str = output_all

equations = []
equation_strs = []  # To keep track of the original equation strings
for eq_str in equations_str:
    expr = sp.sympify(eq_str)
    variables = sorted(expr.free_symbols, key=lambda x: x.name)
    func = lambdify(variables, expr, modules=['numpy'])
    equations.append(func)
    equation_strs.append(eq_str)  # Store the original equation string

def evaluateEquation(individual):
    equation = individual[0]
    y_pred = np.array([equation(x) for x in X[:, 0]])
    y_true = np.array(y)
    error = np.sum((y_true - y_pred) ** 2)
    return (error,)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", random.choice, equations)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.expr, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluateEquation)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutNodeReplacement, pset=None) # Correct mutation function
toolbox.register("select", tools.selTournament, tournsize=3)

# Optimized genetic programming parameters
population_size = 200
num_generations = 100
crossover_prob = 0.7
mutation_prob = 0.3

population = toolbox.population(n=population_size)
result = algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations, verbose=True)

best_ind = tools.selBest(population, 1)[0]
best_index = equations.index(best_ind[0]) # Get the index of the best equation
best_equation_str = equation_strs[best_index] # Retrieve the original equation string

print('Best Equation:', best_equation_str)

## Load fit function

In [ ]:
params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            # beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            beam_size=6
                            )
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms, gp
import sympy as sp
from sympy.utilities.lambdify import lambdify

# Ensure this block runs after obtaining `output_all` from the model
# Create points from an equation
number_of_points = 50
n_variables = 1

# To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"]
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points, len(list(eq_setting["total_variables"]))) * (max_supp - min_supp) + min_supp
X[:, n_variables:] = 0
target_eq = "cos(sin(cos(x_1**2) * sin(x_1)))"  # Use x_1, x_2 and x_3 as independent variables
X_dict = {x: X[:, idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])}
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

output = fitfunc(X, y)
output_all = output["all_bfgs_preds"]
loss_all = output["all_bfgs_loss"]
output_best = output["best_bfgs_preds"]
loss_best = output["best_bfgs_loss"]
print(f"Best prediction: {output_best}")

print(output_all)
equations_str = output_all

equations = []
equation_strs = []  # To keep track of the original equation strings
for eq_str in equations_str:
    expr = sp.sympify(eq_str)
    variables = sorted(expr.free_symbols, key=lambda x: x.name)
    func = lambdify(variables, expr, modules=['numpy'])
    equations.append(func)
    equation_strs.append(eq_str)  # Store the original equation string

def evaluateEquation(individual):
    equation = individual[0]
    y_pred = np.array([equation(x) for x in X[:, 0]])
    y_true = np.array(y)
    error = np.sum((y_true - y_pred) ** 2)
    return (error,)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", random.choice, equations)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.expr, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluateEquation)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutNodeReplacement, pset=None) # Correct mutation function
toolbox.register("select", tools.selTournament, tournsize=3)

# Optimized genetic programming parameters
population_size = 200
num_generations = 100
crossover_prob = 0.7
mutation_prob = 0.3

population = toolbox.population(n=population_size)
result = algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations, verbose=True)

best_ind = tools.selBest(population, 1)[0]
best_index = equations.index(best_ind[0]) # Get the index of the best equation
best_equation_str = equation_strs[best_index] # Retrieve the original equation string

print('Best Equation:', best_equation_str)

In [ ]:
import importlib
import nesymres.architectures.model
importlib.reload(nesymres.architectures.model)
from nesymres.architectures.model import Model


params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            # beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            beam_size=6
                            )
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()
fitfunc = partial(model.fitfunc,cfg_params=params_fit)

In [ ]:
# Create points from an equation
number_of_points = 500
n_variables = 1

#To get best results make sure that your support inside the max and mix support
max_supp = cfg.dataset_train.fun_support["max"] 
min_supp = cfg.dataset_train.fun_support["min"]
X = torch.rand(number_of_points,len(list(eq_setting["total_variables"])))*(max_supp-min_supp)+min_supp
X[:,n_variables:] = 0
target_eq = "cos(sin(cos(x_1**2)*sin(x_1)))" #Use x_1,x_2 and x_3 as independent variables
X_dict = {x:X[:,idx].cpu() for idx, x in enumerate(eq_setting["total_variables"])} 
y = lambdify(",".join(eq_setting["total_variables"]), target_eq)(**X_dict)

output = fitfunc(X,y) 
# output_all = output["all_bfgs_preds"]
# loss_all = output["all_bfgs_loss"]
# output_best = output["best_bfgs_preds"]
# loss_best = output["best_bfgs_loss"]
# print(f"Best prediction: {output_best}")

In [ ]:
print(params_fit.id2word)

## MLP

In [ ]:
from nesymres.architectures.data import DataModule
from pytorch_lightning import Trainer, seed_everything
import hydra
from pathlib import Path

train_path = '/workspaces/NeuralSymbolicRegressionThatScales/data/datasets/10000/'
val_path = '/workspaces/NeuralSymbolicRegressionThatScales/data/raw_datasets/150'

seed_everything(9)
# train_path = Path(hydra.utils.to_absolute_path(train_path))
# val_path = Path(hydra.utils.to_absolute_path(val_path))
data = DataModule(
    train_path,
    val_path,
    None,
    cfg
)

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import torch
import torch.nn.functional as F
from torch.optim import Adam

class MLPModule(pl.LightningModule):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.001):
        super(MLPModule, self).__init__()
        self.save_hyperparameters()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.layers(x)

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=self.hparams.learning_rate)

    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = F.mse_loss(outputs, targets)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = F.mse_loss(outputs, targets)
        self.log('val_loss', loss)
        return loss


In [ ]:
from nesymres.architectures.data import DataModule
from pytorch_lightning import Trainer, seed_everything
import hydra
from pathlib import Path

def train_model():

    train_path = '/workspaces/NeuralSymbolicRegressionThatScales/data/datasets/10000/'
    val_path = '/workspaces/NeuralSymbolicRegressionThatScales/data/raw_datasets/150'

    seed_everything(9)
    # train_path = Path(hydra.utils.to_absolute_path(train_path))
    # val_path = Path(hydra.utils.to_absolute_path(val_path))
    data_module = DataModule(
        train_path,
        val_path,
        None,
        cfg
    )

    input_size = 100
    hidden_size = 50
    output_size = 3
    model = MLPModule(input_size, hidden_size, output_size)

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        filename='best-checkpoint',
        save_top_k=1,
        mode='min'
    )

    # wandb_logger = WandbLogger(project="my_project")
    
    trainer = pl.Trainer(
        max_epochs=50,
        # gpus=1,
        # logger=wandb_logger,
        callbacks=[checkpoint_callback]
    )

    trainer.fit(model, datamodule=data_module)

if __name__ == "__main__":
    train_model()


In [ ]:
# 假设你有一个 datamodule 和 dataloader 已经设置好
data.setup('fit')  # 如果尚未自动调用
train_loader = data.train_dataloader()

first_batch = next(iter(train_loader))
print(type(first_batch))  # 查看数据类型应为 tuple
# print(first_batch.shape)
inputs, targets, whatsthis = first_batch  # 使用下划线忽略额外的信息
print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)
print("Targets shape:", whatsthis.size)

In [ ]:
print("whatsthis shape:", whatsthis)

### Data preparation

In [ ]:
import pandas as pd
import numpy as np
import json
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sklearn.feature_extraction.text import CountVectorizer

path_to_skeletons = '/workspaces/NeuralSymbolicRegressionThatScales/test_set/test_nc.csv'
path_to_constants = '/workspaces/NeuralSymbolicRegressionThatScales/test_set/test_wc.csv'

skeletons_df = pd.read_csv(path_to_skeletons)
constants_df = pd.read_csv(path_to_constants)


In [ ]:
import json

def parse_support(support_str):
    return json.loads(support_str.replace("'", "\""))

skeletons_df['support'] = skeletons_df['support'].apply(parse_support)
constants_df['support'] = constants_df['support'].apply(parse_support)

In [73]:
# print(constants_df)

#### Samples

In [94]:
import numpy as np
import re
from tqdm import tqdm

def extract_sample_points(support_dict, num_points):
    samples = {var: np.random.uniform(bounds['min'], bounds['max'], num_points)
               for var, bounds in support_dict.items()}
    return pd.DataFrame(samples)


# skeleton_samples = skeletons_df['support'].apply(lambda x: extract_sample_points(x, 500))
# constant_samples = constants_df['support'].apply(lambda x: extract_sample_points(x, 500))
def compute_y(df, num_points):
    full_samples = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="Extracting samples"):
        # obtain equation
        equation = parse_expr(row['eq'], transformations='all')
        valid_samples_count = 0
        sample_points_df = pd.DataFrame()
        while valid_samples_count < num_points:
            # obtain samples x
            new_sample_points_df = extract_sample_points(row['support'], num_points - valid_samples_count)
            # compute y, ignore complex y
            new_sample_points_df['y'] = new_sample_points_df.apply(
                lambda vals: equation.evalf(subs=vals.to_dict()), axis=1)
            # print(f"Equation: {equation}")
            # print(f"Sample x: {new_sample_points_df}")
            # print(f"Output of Sample: {new_sample_points_df['y']}")
            # remove complex
            new_sample_points_df = new_sample_points_df[new_sample_points_df['y'].apply(lambda x: x.is_real)]
            new_sample_points_df = new_sample_points_df.dropna(subset=['y'])
            # valid samples
            sample_points_df = pd.concat([sample_points_df, new_sample_points_df])
            
            valid_samples_count = sample_points_df.shape[0]
        # obtain num_points samples
        sample_points_df = sample_points_df.head(num_points)
        full_samples.append(sample_points_df)
    return full_samples

# skeleton_y = compute_y(skeletons_df, 10)
# constant_y = compute_y(constants_df, 10)






In [95]:
# print(constants_df['eq'])
constant_y = compute_y(constants_df, 10)

Extracting samples: 100%|██████████| 100/100 [00:09<00:00, 10.67it/s]


In [96]:
print(constant_y[0])

        x_2       x_1       x_3                  y
0 -2.038740  5.543135  7.350590  -49.4450233301101
1 -9.021707 -2.841883 -6.809667  -49.2126276444300
2 -6.221746  0.189251  1.573105  -2.28853097374882
3  2.832158 -2.453522  2.194735  -44.5415445659957
4  4.297825 -7.403096 -7.731069   501.367948241069
5 -6.972310  3.542027  9.089860  -79.0920486914688
6 -9.475093 -8.817643 -8.215728  -76.3151995122842
7 -6.890810 -5.632550  2.992358  -14.5898025183408
8 -4.818363  0.763671 -3.135211  -9.04054641968066
9  5.964885 -7.801742 -2.611364   1002.52634593528


#### Skeleton embeddings

In [ ]:
import importlib
import nesymres.architectures.model
importlib.reload(nesymres.architectures.model)
from nesymres.architectures.model import Model


params_fit = FitParams(word2id=eq_setting["word2id"], 
                            id2word={int(k): v for k,v in eq_setting["id2word"].items()}, 
                            una_ops=eq_setting["una_ops"], 
                            bin_ops=eq_setting["bin_ops"], 
                            total_variables=list(eq_setting["total_variables"]),  
                            total_coefficients=list(eq_setting["total_coefficients"]),
                            rewrite_functions=list(eq_setting["rewrite_functions"]),
                            bfgs=bfgs,
                            # beam_size=cfg.inference.beam_size #This parameter is a tradeoff between accuracy and fitting time
                            beam_size=2
                            )
## Load architecture, set into eval mode, and pass the config parameters
model = Model.load_from_checkpoint(weights_path, cfg=cfg.architecture)
model.eval()
if torch.cuda.is_available(): 
  model.cuda()
bs_results = partial(model.bs_results,cfg_params=params_fit)

In [ ]:

def generate_model_outputs(sample_dataframes):
    outputs = []
    for df in tqdm(sample_dataframes, desc="Getting skeletons"):
    # for df in sample_dataframes:
        X = df.drop(columns=['y']).values  # x is a vector except y
        y = df['y'].values
        y = np.array(y, dtype=np.float32)
        # print(y)
        output = bs_results(X, y)
        outputs.append(output)
    return outputs

skeleton_outputs = generate_model_outputs(skeleton_y)
constant_outputs = generate_model_outputs(constant_y)

In [66]:
def pad_sequences(sequences, maxlen, dtype='int', padding='post', truncating='post', value=0):
    """Pad each sequence to the same length: the length of the longest sequence."""
    padded_sequences = np.full((len(sequences), maxlen), fill_value=value, dtype=dtype)
    for i, seq in enumerate(sequences):
        if len(seq) > maxlen:
            if truncating == 'pre':
                truncated = seq[-maxlen:]
            else:
                truncated = seq[:maxlen]
        else:
            truncated = seq
        if padding == 'post':
            padded_sequences[i, :len(truncated)] = truncated
        else:
            padded_sequences[i, -len(truncated):] = truncated
    return padded_sequences

# Extract tensors from the skeleton_outputs, ignoring scores
tensors = [output[0][1].numpy() for output in skeleton_outputs]  # Assuming these are PyTorch tensors

# Pad the extracted tensors to a maximum length of 20
max_length = 20
padded_tensors = pad_sequences(tensors, maxlen=max_length, dtype='int')

print("Padded Tensors:")
print(padded_tensors)

Padded Tensors:
[[ 1 18  6 ...  0  0  0]
 [ 1  9 18 ...  0  0  0]
 [ 1 20 18 ...  0  0  0]
 ...
 [ 1  9 19 ...  0  0  0]
 [ 1  9  5 ...  0  0  0]
 [ 1 18  6 ...  0  0  0]]


In [104]:
print(constant_y[0])

        x_2       x_1       x_3                  y
0 -2.038740  5.543135  7.350590  -49.4450233301101
1 -9.021707 -2.841883 -6.809667  -49.2126276444300
2 -6.221746  0.189251  1.573105  -2.28853097374882
3  2.832158 -2.453522  2.194735  -44.5415445659957
4  4.297825 -7.403096 -7.731069   501.367948241069
5 -6.972310  3.542027  9.089860  -79.0920486914688
6 -9.475093 -8.817643 -8.215728  -76.3151995122842
7 -6.890810 -5.632550  2.992358  -14.5898025183408
8 -4.818363  0.763671 -3.135211  -9.04054641968066
9  5.964885 -7.801742 -2.611364   1002.52634593528


In [130]:
all_samples_df = pd.concat(constant_y, ignore_index=True)
# print(all_samples_df)
# obtain features
X_features = all_samples_df.drop(columns=['y']).values  
y_features = all_samples_df['y'].values
y_features = np.array(y_features, dtype=np.float32)

# padded_tensors is the output of transformers
padded_tensors = np.array(padded_tensors)

if padded_tensors.shape[0] != len(constant_y):
    raise ValueError("Number of samples in constant_y does not match number of samples in padded_tensors")

X_data = []
y_data = []

# iterate all samples
for i, tensor in tqdm(enumerate(padded_tensors), total=len(padded_tensors), desc="Combining embeddings and data points"):
    sample_df = constant_y[i]
    sample_features = sample_df.drop(columns=['y']).values
    sample_y = sample_df['y'].values
    sample_y = np.array(sample_y, dtype=np.float32)
    combined_features = tensor
    # combine embedding, sample_features and sample_y
    for j in range(10):
        max_length_feature = 23
        x_feature = sample_features[j]
        if x_feature.shape[0] < max_length_feature:
            padding_length = max_length_feature - x_feature.shape[0]
            x_feature = np.pad(x_feature, (0, padding_length), 'constant')
        combined_features = np.hstack([combined_features, x_feature])
        # print(f"Feature shape in x :{combined_features.shape}"
        
        # if combined_features.shape[0] < max_length_feature:
            # padding_length = max_length_feature - combined_features.shape[0]
            # combined_features = np.pad(combined_features, (0, padding_length), 'constant')
        combined_features = np.hstack([combined_features,sample_y[j]])
    
        # print(f"Shape: {tensor.shape}, {sample_features.shape}, {sample_y.shape}")
        # print(combined_features.shape)
        # y_data.append(sample_y[j])
    X_data.append(combined_features)
    # print(f"X shape: {combined_features.shape}")

X_data = np.array(X_data)
# y_data = np.array(y_data)

print("Combined features shape:", X_data.shape)
# print("Target shape:", y_data.shape)

/tmp/ipykernel_15192/1354399185.py:6: RuntimeWarning: overflow encountered in cast
  y_features = np.array(y_features, dtype=np.float32)
Combining embeddings and data points:   0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipykernel_15192/1354399185.py:22: RuntimeWarning: overflow encountered in cast
  sample_y = np.array(sample_y, dtype=np.float32)
Combining embeddings and data points: 100%|██████████| 100/100 [00:00<00:00, 1605.57it/s]

Combined features shape: (100, 260)


In [ ]:
print(skeleton_outputs[0])

In [74]:
constant_y = compute_y(constants_df, 10)
print("Padded Tensors shape:", padded_tensors.shape)
print("Constant Y shape:", constant_y[0])


Extracting samples: 100%|██████████| 100/100 [00:09<00:00, 10.08it/s]

Padded Tensors shape: (100, 20)
Constant Y shape:         x_2       x_1       x_3                  y
0 -4.787315 -7.000346  7.448065  -62.5360934657640
1 -2.362670  1.560538 -1.911053  -1.91162257703467
2 -2.583387 -0.106637 -3.326650  -10.9220133750830
3  6.529477  3.113005 -2.213097   1514.27401881362
4 -5.464418  4.863633 -0.713338   4.35780322806024
5 -7.047403  2.984540  0.416318   2.81085770988115
6  5.239405  7.332776 -7.780136   1413.80814146908
7 -7.605424  4.063913  7.956521  -59.2462666225795
8 -1.686573 -7.093847  5.059850  -33.6327789085535
9  0.690136  0.620391 -0.039348  0.697302158187894


#### Label

In [139]:
def extract_constants(equation):
    constants = []
    for atom in equation.atoms(sp.Number):
        if atom != sp.S.One:
            constants.append(float(atom))
    return constants

# extract constants of equation.
def get_constants_labels(df):
    labels = []
    for i, row in df.iterrows():
        eq_str = row['eq']
        equation = parse_expr(eq_str, transformations='all')
        constants = extract_constants(equation)
        labels.append(constants)
    return labels

constant_labels = get_constants_labels(constants_df)
constant_labels_array = np.array(constant_labels, dtype=object)

print("Constant Labels Shape:", constant_labels_array.shape)
max_length = max(len(label) for label in constant_labels)
padded_labels = np.array([np.pad(label, (0, max_length - len(label)), 'constant') for label in constant_labels])

# print out the output
print("Padded Constant Labels Shape:", padded_labels.shape)
# print("Padded Constant Labels:", padded_labels)


Constant Labels Shape: (100,)
Padded Constant Labels Shape: (100, 5)


### Model

In [152]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# a simple regression model
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [141]:
def prepare_dataloader(X, y, batch_size=32):
    dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader


In [149]:
def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=1e-3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")






In [150]:
print(X_data.shape)
print(padded_labels.shape)

(100, 260)
(100, 5)


In [154]:
# data prepare
train_loader, val_loader = prepare_dataloader(X_data, padded_labels, batch_size=32)

# model initialization
input_dim = X_data.shape[1]
output_dim = padded_labels.shape[1]
model = RegressionModel(input_dim, output_dim)
# train model
train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=1e-1)

Epoch 1/50, Training Loss: nan, Validation Loss: nan
Epoch 2/50, Training Loss: nan, Validation Loss: nan
Epoch 3/50, Training Loss: nan, Validation Loss: nan
Epoch 4/50, Training Loss: nan, Validation Loss: nan
Epoch 5/50, Training Loss: nan, Validation Loss: nan
Epoch 6/50, Training Loss: nan, Validation Loss: nan
Epoch 7/50, Training Loss: nan, Validation Loss: nan
Epoch 8/50, Training Loss: nan, Validation Loss: nan
Epoch 9/50, Training Loss: nan, Validation Loss: nan
Epoch 10/50, Training Loss: nan, Validation Loss: nan
Epoch 11/50, Training Loss: nan, Validation Loss: nan
Epoch 12/50, Training Loss: nan, Validation Loss: nan
Epoch 13/50, Training Loss: nan, Validation Loss: nan
Epoch 14/50, Training Loss: nan, Validation Loss: nan
Epoch 15/50, Training Loss: nan, Validation Loss: nan
Epoch 16/50, Training Loss: nan, Validation Loss: nan
Epoch 17/50, Training Loss: nan, Validation Loss: nan
Epoch 18/50, Training Loss: nan, Validation Loss: nan
Epoch 19/50, Training Loss: nan, Vali